# Preprocessing the Yelp dataset
## Preprocess the business.json and merge it with review.json

In [92]:
import pandas as pd

In [93]:
business_json_path = '/Users/zhiyuwang/Desktop/Desktop/tmp/yelp_dataset/business.json'
review_json_path = '/Users/zhiyuwang/Desktop/Desktop/tmp/yelp_dataset/review.json'

In [94]:
df_b=pd.read_json(business_json_path, lines=True)

**Clean the business.json file**
* Include only current opened restaurants
* we only want to focus on the restaurants data

In [96]:
# 1= open, 0 = closed
df_b = df_b[df_b['is_open']==1]

# Only focus on restaurants and food
df_b = df_b[df_b['categories'].str.contains('Restaurants|Food', case=False, na=False)]

In [97]:
# all columns in the business
for col in df_b.columns:
    print(col)

address
attributes
business_id
categories
city
hours
is_open
latitude
longitude
name
postal_code
review_count
stars
state


**Drop some irrelevant columns**
* As we are curiously about geo info of the restaurants, so hours, is_open, review_counts can be dropped.
* We will need to keep the business_id to merge with the reviews.json

In [98]:
drop_columns = ['hours','is_open','state', 'postal_code']
df_b = df_b.drop(drop_columns, axis=1)

**not all cities have enough records, so we would mainly focus on the top 2 cities with most restaurants.**

In [99]:
df_b.city.value_counts()[:2]

Toronto      6847
Las Vegas    5621
Name: city, dtype: int64

In [102]:
vegas_business = df_b[df_b['city'].str.contains('Las Vegas', case=False, na=False)]
vegas_business.to_csv("./vegas/vegas_business.csv", index=False)

toronto_business = df_b[df_b['city'].str.contains('Toronto', case=False, na=False)]
toronto_business.to_csv("./toronto/toronto_business.csv", index=False)

## Review.json is a huge file, loading all data at once will easily take all memory space on you machine, so it would be very necessary to load the data in chunks.

In [103]:
size = 1000000
review = pd.read_json(review_json_path,lines = True, chunksize = size)                   

In [105]:
vegas_chunk_list = []
toronto_chunk_list = []
drop_columns = ['review_id', 'user_id','funny','cool']
for chunk_review in review:
    chunk_review = chunk_review.drop(['review_id','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    vegas_chunk_merged = pd.merge(vegas_business, chunk_review, on='business_id', how='inner')
    toronto_chunk_merged = pd.merge(toronto_business, chunk_review, on='business_id', how='inner')
    print(f"on vegas: {vegas_chunk_merged.shape[0]} out of {size:,} related reviews")
    print(f"on toronto: {toronto_chunk_merged.shape[0]} out of {size:,} related reviews")
    vegas_chunk_list.append(vegas_chunk_merged)
    toronto_chunk_list.append(toronto_chunk_merged)
    
vegas_df = pd.concat(vegas_chunk_list, join='outer',axis=0)
toronto_df = pd.concat(toronto_chunk_list, join='outer', axis=0)

on vegas: 176413 out of 1,000,000 related reviews
on toronto: 52143 out of 1,000,000 related reviews
on vegas: 174232 out of 1,000,000 related reviews
on toronto: 47978 out of 1,000,000 related reviews
on vegas: 155655 out of 1,000,000 related reviews
on toronto: 52972 out of 1,000,000 related reviews
on vegas: 180634 out of 1,000,000 related reviews
on toronto: 52201 out of 1,000,000 related reviews
on vegas: 167825 out of 1,000,000 related reviews
on toronto: 51117 out of 1,000,000 related reviews
on vegas: 124741 out of 1,000,000 related reviews
on toronto: 38016 out of 1,000,000 related reviews


In [106]:
vegas_csv_name = "./vegas/vegas_business+review.csv"
vegas_df.to_csv(vegas_csv_name, index=False)

In [107]:
toronto_csv_name = "./toronto/toronto_business+review.csv"
toronto_df.to_csv(toronto_csv_name, index=False)

# Conclusion:
* So far we have finished fundamental data preprocessing for business.json and review.json, and we merged these 2 files together based on business_id of restaurants. We would make latter analysis and visualization easiler and more effective.
* Data is useful only when we can read and store them properly. After the preprocessing, we get 4 "clean" data files as output (2 for Las vegas, 2 for Toronto).
* 